# Extended Rental Price Prediction with Visualizations and Borough Selector


This notebook extends the multiple linear regression analysis to:
- Visualize the impact of features on rental price.
- Include data from Queens and Brooklyn alongside Manhattan.
- Provide an interactive interface for predicting rental prices.


In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

In [2]:
streeteasy_manhattan = pd.read_csv("https://raw.githubusercontent.com/Codecademy/datasets/master/streeteasy/manhattan.csv")
streeteasy_brooklyn = pd.read_csv("https://raw.githubusercontent.com/Codecademy/datasets/master/streeteasy/brooklyn.csv")
streeteasy_queens = pd.read_csv("https://raw.githubusercontent.com/Codecademy/datasets/master/streeteasy/queens.csv")

datasets = {
    "Manhattan": streeteasy_manhattan.copy(),
    "Brooklyn": streeteasy_brooklyn.copy(),
    "Queens": streeteasy_queens.copy()
}

In [3]:
def train_rent_model(df):
    features = ['bedrooms', 'bathrooms', 'size_sqft', 'min_to_subway', 'floor',
                'building_age_yrs', 'no_fee', 'has_roofdeck', 'has_washer_dryer',
                'has_doorman', 'has_elevator', 'has_dishwasher', 'has_patio', 'has_gym']
    X = df[features]
    y = df['rent']
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
    
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    return model, features

In [4]:
models = {}
feature_sets = {}

for borough, df in datasets.items():
    model, features = train_rent_model(df)
    models[borough] = model
    feature_sets[borough] = features

In [5]:
def create_predictor(borough):
    clear_output(wait=True)
    display(borough_selector)

    model = models[borough]
    features = feature_sets[borough]
    
    input_widgets = {
        'bedrooms': widgets.IntSlider(value=1, min=0, max=5),
        'bathrooms': widgets.FloatSlider(value=1.0, min=0.5, max=4.0, step=0.5),
        'size_sqft': widgets.IntSlider(value=500, min=200, max=2000),
        'min_to_subway': widgets.IntSlider(value=5, min=0, max=30),
        'floor': widgets.IntSlider(value=1, min=0, max=50),
        'building_age_yrs': widgets.IntSlider(value=10, min=0, max=100),
        'no_fee': widgets.Checkbox(value=True),
        'has_roofdeck': widgets.Checkbox(value=False),
        'has_washer_dryer': widgets.Checkbox(value=False),
        'has_doorman': widgets.Checkbox(value=False),
        'has_elevator': widgets.Checkbox(value=True),
        'has_dishwasher': widgets.Checkbox(value=False),
        'has_patio': widgets.Checkbox(value=False),
        'has_gym': widgets.Checkbox(value=False)
    }

    ui = widgets.VBox([widgets.HBox([widgets.Label(k), v]) for k, v in input_widgets.items()])
    display(ui)

    def predict_rent(**kwargs):
        cleaned_input = {k: int(v) if isinstance(v, bool) else v for k, v in kwargs.items()}
        X_input = pd.DataFrame([[cleaned_input[f] for f in features]], columns=features)
        prediction = model.predict(X_input)[0]
        print(f"Predicted Rent for {borough}: ${prediction:.2f}")

    out = widgets.interactive_output(predict_rent, input_widgets)
    display(out)


borough_selector = widgets.Dropdown(options=datasets.keys(), description='Borough:')
display(borough_selector)

create_predictor(borough_selector.value)

def on_change(change):
    if change['name'] == 'value':
        create_predictor(change['new'])

borough_selector.observe(on_change)
borough_selector = widgets.Dropdown(options=datasets.keys(), description='Borough:')
display(borough_selector)

create_predictor(borough_selector.value)

def on_change(change):
    if change['name'] == 'value':
        create_predictor(change['new'])

borough_selector.observe(on_change)

Dropdown(description='Borough:', options=('Manhattan', 'Brooklyn', 'Queens'), value='Manhattan')

Output()